# CH 12 DIMENSION REDUCTION


# HANDS-ON ANALYSIS

---
# Use **cereals** Dataset below.

---

#21
Standardize or normalize the predictors Sugars, Fiber, and Potass.


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
folder = "/gdrive/My Drive/Python Practice/Datasets"

import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.outliers_influence as inf
#PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

cereals = pd.read_csv(folder + '/cereals.CSV')
cereals = cereals.dropna()
#np.isnan(cereals['Sugars']).sum()
#np.isnan(cereals['Fiber']).sum()
#np.isnan(cereals['Potass']).sum()

cols = ['Sugars', 'Fiber', 'Potass']
cereals_z = pd.DataFrame(stats.zscore(cereals[cols]), columns=cols)
cereals_z.head()

#22
Construct the correlation matrix for Sugars, Fiber, and Potass. <br>
Which variables are highly correlated?

In [ ]:
X = pd.DataFrame(cereals[cols], columns = cols)
Y = pd.DataFrame(cereals[['Rating']])

pd.plotting.scatter_matrix(X, figsize=(10,10))
X.corr()
#Potass and Fiber are highly correlated

#23
Build a regression model to estimate Rating based on Sugars, Fiber, and Potass.<br>
Obtain the VIFs from the model. <br>
Which VIFs indicate that multicollinearity is a problem?


In [ ]:
X = sm.add_constant(cereals[cols])
Y = pd.DataFrame(cereals[['Rating']])
Y.index = X.index

m01 = sm.OLS(Y, X).fit()
print(m01.summary()) 

In [ ]:
# VIF 
[inf.variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
# Since VIF of Fiber, Potass > 5, it has moderate multicollinearity

#24
Run PCA using varimax rotation and three components. <br>
What percent of the variability is explained by one component? <br>
By two components? <br>
By all three components?


In [ ]:
!pip install factor-analyzer

In [ ]:
# varimax rotation

from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(n_factors=3, method='principal', rotation="varimax")
fa.fit(cereals[cols])

#print(fa.loadings_.round(4))       # meaning?    ex) 1st row : [-0.0278  0.9995 -0.0149] means that Component 1 contains Fiber factor (0.9995 > 0)
#fa.get_eigenvalues()               # meaning?

fa.get_factor_variance()            # 1st row : eigenvalue / 2nd row : variance per each factor / 3rd row : cumulative variance per each factor

# One component explains 63.6 % of variability
# Two Components explain 97.4% of variability 

In [ ]:
#no rotation

X = pd.DataFrame(cereals[cols], columns = cols)
Y = pd.DataFrame(cereals[['Rating']])

pca01 = PCA(n_components=3)
principComp = pca01.fit_transform(X)    # principComp meaning ?

print(pca01.explained_variance_ratio_)              # variability per component 1,2,3
print(np.cumsum(pca01.explained_variance_ratio_))   # cumulative variability

# Component 1 explains 99 % of variability

#25
Make a plot of the eigenvalues of the three components. <br>
Using the eigenvalue criterion, how many components would you retain?

In [ ]:
import matplotlib.pyplot as plt

cov_mat = np.cov(cereals_z, rowvar=False)
evals = np.linalg.eigvals(cov_mat)              # eigen value of covariance matrix... meaning?
comp = [1,2,3]

plt.plot(comp, evals, color='black')
plt.xlabel("Component")
plt.ylabel("Eigenvalues")
plt.xticks(np.arange(1,4,1))
plt.show()

#26
Say we want to explain at least 70% of the variability. <br>
How many components would you retain?


In [ ]:
# eigenvalue from numpy package 
cov_mat = np.cov(X, rowvar=False)
evals = np.linalg.eigvals(cov_mat)  
evals
# Based on Eigenvalue Criterion (if eigen > 1, would retain that component) , I would retain ONE COMPONENT  

In [ ]:
# eigenvalue from Factor Analyzer package
fa.get_eigenvalues()
# Based on Eigenvalue Criterion, TWO COMPONENTS will be retained

#27
Run PCA using varimax rotation and two components. <br>
What percent of the variability do the two components explain?

In [ ]:
# Using two components, run PCA

from factor_analyzer import FactorAnalyzer

fa = FactorAnalyzer(n_factors=2, method='principal', rotation="varimax")
fa.fit(cereals[cols])

fa.get_factor_variance()  
# Two components explain 97.4% of variability 

In [ ]:
#no rotation
pca01 = PCA(n_components=2)
principComp = pca01.fit_transform(X)

pca01.explained_variance_ratio_ 
# One component explains 99.6% of variability if No rotation

#28
What variable or variables are contained in Component 1? What variable or variables are contained in Component 2?

In [ ]:
fa = FactorAnalyzer(n_factors=3, method='principal', rotation="varimax")
fa.fit(cereals[cols])

print(fa.loadings_.round(4)) 

# Components 1 : Fiber (0.9995 > 0)
# Components 2 : Sugars (0.9708 > 0), Potass (0.2074 > 0)

#29
Use the two components as the predictor variables in a regression model to estimate Rating. <br>
What are the regression coefficients of the two components?


In [ ]:
from sklearn.linear_model import LinearRegression

X_new = fa.transform(X)
X_new = pd.DataFrame(X_new, columns=['P1','P2','P3'])
Y_new = pd.DataFrame(cereals['Rating'])

X_pca = X_new.iloc[:,:2]

reg = LinearRegression().fit(X_pca, Y_new)
reg.coef_

In [ ]:
# Linear Regression (from statsmodel pacakge) 
import statsmodels.api as sm

Y_new.index = X_pca.index

reg1 = sm.OLS(Y_new, X_pca).fit()
reg1.summary()
# coef [p1 : 6.7411, p2: -10.3105]

#30
What are the VIFs of the two components in the regression model?


In [ ]:
# VIF 
[inf.variance_inflation_factor(X_pca.values,i) for i in range(X_pca.shape[1])] 
# VIF is very close to 1. Good
# VIF = 1 / (1 - R^2) 

---
# Use **red_wine_PCA_training**, **red_wine_PCA_test** Dataset below.

---

In [ ]:
red_train = pd.read_csv(folder + "/red_wine_PCA_training")
red_test = pd.read_csv(folder + "/red_wine_PCA_test")

red_train = red_train.dropna()
red_test = red_test.dropna()

Y = red_train[['quality']]
X_names = ['alcohol', 'residual sugar', 'pH', 'density', 'fixed acidity']
X = pd.DataFrame(red_train[X_names])

#31
Standardize or normalize the predictors.

In [ ]:
red_z = pd.DataFrame(stats.zscore(X), columns=X_names)
red_z.head()

#32
Construct the correlation matrix for the predictors. <br>
Between which predictors do you find the highest correlations?


In [ ]:
X.corr()
# pH and fixed acidity has the highest correlation (66%)

#33
Build a regression model to estimate quality based on the predictors. <br>
Obtain the VIFs from the model. <br>
Which VIFs indicate that multicollinearity is a problem? <br>
Compare the variables with high VIF to the correlated variables from the previous exercise.


In [ ]:
# Linear Regression (from sklearn pacakge) 
from sklearn.linear_model import LinearRegression

X_const = sm.add_constant(X)
reg = LinearRegression().fit(X_const, Y)
reg.coef_

In [ ]:
# Linear Regression (from statsmodel pacakge) 
import statsmodels.api as sm

X_const = sm.add_constant(X)
reg1 = sm.OLS(Y, X_const).fit()
reg1.summary()

In [ ]:
# VIF 
X_const = sm.add_constant(X)
[inf.variance_inflation_factor(X_const.values,i) for i in range(X_const.shape[1])] 
# (VIF_density, VIF_fixed acidity) have high VIF
# Corr(density, fixed acidity) also has the highest (0.6466)

#34
Perform PCA using varimax rotation. <br>
Show the rotated proportions of variance explained for extracting up to five components.<br>
What percent of the variability is explained by one component? <br>
By two components? <br>
By three components? <br>
By four components? <br>
By all five components?


In [ ]:
# varimax rotation

from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(n_factors=5, method='principal', rotation="varimax")
fa.fit(X)

#print(fa.loadings_.round(4))       # meaning?    ex) 1st row : [-0.0278  0.9995 -0.0149] means that Component 1 contains Fiber factor (0.9995 > 0)
#fa.get_eigenvalues()               # meaning?

fa.get_factor_variance()            # 1st row : eigenvalue / 2nd row : variance per each factor / 3rd row : cumulative variance per each factor

# One component explains 29.21 % of variability
# Two Components explain 53.79% of variability 
# Three Components explain 77.01% of variability 
# Four Components explain 97.87% of variability 
# All five Components explain 100.00% of variability 

In [ ]:
#no rotation

pca02 = PCA(n_components=5)
principComp = pca02.fit_transform(X)

print(pca02.explained_variance_ratio_)              # variability per components
print(np.cumsum(pca02.explained_variance_ratio_))   # cumulative variability

# 3 components explain 99.79 % of variability

#35
Say we want to explain at least 90% of the variability. <br>
How many components does the proportion of variance explained criterion suggest we extract?


In [ ]:
# 4 components 

#36
Make a plot of the eigenvalues of the five components. <br>
According to the eigenvalue cri- terion, how many components should we extract?


In [ ]:
import matplotlib.pyplot as plt

cov_mat = np.cov(red_z, rowvar=False)
evals = np.linalg.eigvals(cov_mat)              # eigen value of covariance matrix... meaning?
comp = [1,2,3,4,5]

plt.plot(comp, evals, color='black')
plt.xlabel("Component")
plt.ylabel("Eigenvalues")
plt.xticks(np.arange(1,6,1))
plt.axhline(y=1.0, color='r',linestyle='dashed')
plt.show()

# 3 components should be extracted based on the eigenvalue criterion

#37
Combine the recommendations from the two criteria to reach a consensus as to how many components we should extract.


In [ ]:
# The Proportion of Variance Explained Criterion : Extract 4 components 
# The Eigenvalue Criterion : Extract 3 components
# Combine these two criterion : Extract 4 components

#38
Profile each of your components, stating which variables are included, and noting their within‐component correlation (positive or negative). <br>
For simplicity, consider compo- nents weights greater than 0.5 only.


In [ ]:
# varimax rotation

from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(n_factors=5, method='principal', rotation="varimax")
fa.fit(X)

print(fa.loadings_.round(4))       # meaning?    ex) 1st row : [-0.0278  0.9995 -0.0149] means that Component 1 contains Fiber factor (0.9995 > 0)
#fa.get_factor_variance()            # 1st row : eigenvalue / 2nd row : variance per each factor / 3rd row : cumulative variance per each factor

# Principal Component 1 contains pH 
# Principal Component 2 contains density
# Principal Component 3 contains residual sugar
# Principal Component 4 contains alcohol
# Principal Component 5 contains alcohol

In [ ]:
#pca.components_

#39
Produce the correlation matrix for the components. <br>
What do these values mean?


In [ ]:
pca02 = PCA(n_components=5)
X_pca = pca02.fit_transform(X)
X_pca = pd.DataFrame(X_pca, columns=X_names)
X_pca.corr()

# No correlation between predictor variables.
# orthogonal to each other

#40
Next, use only the components you extracted to estimate wine quality using a regression model. <br>
Do not include the original predictors.<br>
a.  Compare the values of s and R_adj between the PCA regression and the original regres- sion model.<br>
b.  Explain why the original model slightly outperformed the PCA model.<br>
c.  Explain how the PCA model may be considered superior, even though slightly outperformed?


In [ ]:
from sklearn.linear_model import LinearRegression

X_new = fa.transform(X)
X_new = pd.DataFrame(X_new, columns=['P1','P2','P3', 'P4', 'P5'])
Y_new = pd.DataFrame(red_train['quality'])

X_pca = X_new.iloc[:,:4]

reg = LinearRegression().fit(X_pca, Y_new)
reg.coef_

In [ ]:
# Linear Regression (from statsmodel pacakge) 
import statsmodels.api as sm

Y_new.index = X_pca.index

reg2 = sm.OLS(Y_new, X_pca).fit()

print("PCA Regression Model\ns: %.4f \nR: %.4f" % (np.sqrt(reg2.scale), reg2.rsquared_adj) )  # s, R_adj 
print(reg2.summary())  # summary 
# R: -0.0003
# s: 5.6766

In [ ]:
# Original Regression model 
reg_orig = sm.OLS(Y, X).fit()

print("Original Regression Model\ns: %.4f \nR: %.4f" % (np.sqrt(reg_orig.scale), reg_orig.rsquared_adj) )  # s, R_adj 
print(reg_orig.summary())  # summary
# s: 0.6851
# R: 0.986

In [ ]:
# Why original model outperform ?   s of original model is very smaller than PCA model

#41
In your regression from the previous exercise, what are the VIFs of the two components in the regression model? What do these values mean?


In [ ]:
X_pca.head()

In [ ]:
# VIF 
[inf.variance_inflation_factor(X_pca.values,i) for i in range(X_pca.shape[1])] 
# VIF is very close to 1. Good
# VIF = 1 / (1 - R^2) 